<a href="https://colab.research.google.com/github/don05050505/don05050505/blob/main/LSTMKorean_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install Korpora
!pip install konlpy

In [ ]:
from konlpy.tag import Hannanum
from Korpora import Korpora
import pandas as pd
import numpy as np

In [ ]:
Korpora.fetch('nsmc')
corpus = Korpora.load('nsmc')

In [ ]:
train_text = corpus.train.texts
train_labels = corpus.train.labels
test_text = corpus.test.texts
test_labels = corpus.test.labels
Xtrain = pd.DataFrame(train_text, columns=['text'])
ytrain = pd.DataFrame(train_labels, columns=['labels'])
train_data = pd.concat([Xtrain, ytrain], axis=1) # x축(axis=1)
Xtest = pd.DataFrame(test_text, columns=['text'])
ytest = pd.DataFrame(test_labels, columns=['labels'])
test_data = pd.concat([Xtest, ytest], axis=1) # x축(axis=1)
print(test_data)

In [ ]:
def get_train_test_data(train, test):
    XTrain = train['text'].values
    YTrain = train['labels'].values
    XTest = test['text'].values
    YTest = test['labels'].values

    return XTrain, YTrain, XTest, YTest

In [ ]:
X_train, y_train, X_test, y_test = get_train_test_data(train_data, test_data)
print(X_train[0], y_train[0])

In [ ]:
hannanum = Hannanum()

words = hannanum.morphs(X_train[0])
print(X_train[0], words, y_train[0])

In [ ]:
strop_words = ['..', '...', '....', '요', '에', '에서', '은', '는', '이', '가', '?', '.', '을', '를',\
               '이다', '게', '에게', '의', '로', ',' , '으로']
XT = X_test[:1000]
print(type(trainData))

def refine(trainData):
    maxCnt = 0
    for i, train in enumerate(trainData):
        words = hannanum.morphs(train) # list

        for word in words:
            if word in strop_words:
                words.remove(word)

        if len(words)>maxCnt: maxCnt = len(words)
        string = ' '.join(words)
        trainData[i] = string
    return trainData, maxCnt



In [ ]:
#  한글 작업을 할 경우
trainData, maxCnt = refine(XT)
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
vector_layer = TextVectorization(
        max_tokens = 200000,
        output_mode = 'int',
        output_sequence_length = maxCnt+1)
vector_layer.adapt(tf.cast(trainData, tf.string)) # Bag of words
vocab_set = sorted(vector_layer.get_vocabulary())
vocab_size = len(vocab_set)
print(vocab_set)

In [ ]:
embedding_dim = 1000
sequence_length = 30
lstm_length = 256
model = tf.keras.Sequential()
model.add(tf.keras.layers.Input(shape=(1, ), dtype=tf.string))
model.add(vector_layer)
model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=sequence_length))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM((lstm_length))))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid')) # 이진분류일 때
# model.add(tf.keras.layers.Dense(num_labels, activation='softmax')) # 다진분류일 때
model.compile(optimizer='RMSprop', loss='binary_crossentropy', metrics=['accuracy']) # 이진분류일 때
# model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(), metrices=['accuracy']) # 다진분류일 때

In [ ]:
X = trainData
y = y_test[:1000]
print(X[5], y[5])

In [ ]:
model.fit(X, y, epochs = 10, validation_split=0.2, batch_size=30,  verbose=1, shuffle=True)

In [ ]:
Xtest = X_test[1000:1100]
ytest = y_test[1000:1100]

xt, _ = refine(Xtest)
print(xt)

In [ ]:
model.evaluate(xt, ytest, batch_size=10)

In [ ]:
pred = model.predict(xt)
# preds = []
# for p in pred:
#     if p > 0.5:
#         val =1
#     else:
#         val = 0
#     preds.append(val)
# print(preds)
# print(ytest)
print(pred)